In [ ]:
Temp = np.array([530,580,630,650,680])
x = -1000/(np.array(Temp))
JV_exp =np.loadtxt('/mnt/c/Users/pssrg/Downloads/GaAs_exp_nJV.txt')
par = np.loadtxt("/mnt/c/Users/pssrg/Downloads/GaAs_sim_label.txt")
plt.plot(JV_exp[0,:])
plt.xlabel('voltage (a.u.)')
plt.ylabel('current (a.u.)')

In [ ]:
def log_norm_pdf(y,mu,sigma):
    return -0.5*np.sum((y-mu)**2/sigma)+np.log(sigma)

#define the logprobability based on Arrhenius equation 

def log_probability(theta,x,y,sigma):
    a1,b1, c1, a2,b2,c2, a3,b3,c3, a4,b4,c4, a5,b5,c5 = theta
    emitter_doping = a1*np.log(-1/x)+b1*x+c1
    back_doping = a2*np.log(-1/x)+b2*x+c2
    tau = (a3*np.log(-1/x)+b3*x+c3)
    fsrv = (a4*np.log(-1/x)+b4*x+c4)
    rsrv = (a5*np.log(-1/x)+b5*x+c5)
       
    #stack all 5 materail descriptors
    par_input = 10*np.stack((emitter_doping,back_doping,tau,fsrv,rsrv),axis=-1)
    coeff = [a1,b1,c1,a2,b2,c2,a3,b3,c3,a4,b4,c4,a5,b5,c5]
    
    #setting prior and constraints
    if all(-10<x<10 for x in coeff) and max(np.abs(coeff[0::3]))<5:
    
        if np.max(par_input)<1 and np.min(par_input)>0:
            sim_curves= reg.predict(par_input)       
        
            return log_norm_pdf(sim_curves, y,sigma)
        return -np.inf
    return -np.inf
    
def logp(x):
    return 0.0 
def log_prob(theta):
    return log_probability(theta, x, JV_exp, sigma)

In [ ]:
par = log10_ln(par)


#Normalize JV descriptors column-wise
scaler = MinMaxScaler()

par_n = scaler.fit_transform(par)   


In [ ]:
###########################---First-Chain---###############################

In [ ]:
ndim = 15          
nwalkers = 64      
nruns = 2000  
sigma = 1e-6        

initial = np.tile([0, 0, 0.5], 5)/10 + 1e-4*np.random.randn(nwalkers, ndim)

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob)

sampler.run_mcmc(initial, nruns, progress=True)
samples = sampler.get_chain()             # Shape: (nruns, nwalkers, ndim)
#flat_samples = sampler.get_chain(discard=100, thin=10, flat=True)  

In [ ]:
import pickle
with open("sampler.pkl", "wb") as f:
    pickle.dump(sampler, f)

In [ ]:
import pickle
with open("sampler.pkl", "rb") as f:
    sampler = pickle.load(f)
    samples = sampler.get_chain()   

In [ ]:
###########################---Second-Chain---###############################

In [ ]:
pos_update = good_walkers[-1, :, :] + 1e-5 * np.random.randn(35, 15)

extra_walker = pos_update[0] + 1e-4 * np.random.randn(1, pos_update.shape[1])
pos_update = np.concatenate([pos_update, extra_walker], axis=0)  
# Reset and run second chain
sampler_ = emcee.EnsembleSampler(36, 15, log_prob)
sampler_.run_mcmc(pos_update, 1000, progress=True)

In [ ]:
with open("sampler_.pkl", "wb") as f:
    pickle.dump(sampler, f)

In [ ]:
import pickle
with open("sampler_.pkl", "rb") as f:
    sampler_ = pickle.load(f)
    samples = sampler_.get_chain()             

In [ ]:
###########

In [ ]:
#samples = sampler.get_chain()  # shape: (nruns, nwalkers, ndim)
samples = sampler.get_chain()
flat_samples = sampler.get_chain(flat=True)  # shape: (nruns * nwalkers, ndim)
log_probs = sampler.get_log_prob()  # shape: (nruns, nwalkers)
flat_log_probs = sampler.get_log_prob(flat=True)  # shape: (nruns * nwalkers,)
zero_flat_samples = flat_samples[:,:]
zero_samples = samples[:,:,:]
zero_flat_loss = sampler.lnprobability[:,:]
plt.figure()
plt.plot(-1*log_probs[:,1])
plt.xlabel('run number')
plt.ylabel('loss')

In [ ]:
def check_plot(theta,x,sim):
    
    a1,b1,c1, a2,b2,c2, a3,b3,c3, a4,b4,c4, a5,b5,c5 = theta
    emitter_doping = a1*np.log(-1/x)+b1*x+c1
    back_doping = a2*np.log(-1/x)+b2*x+c2
    tau = (a3*np.log(-1/x)+b3*x+c3)
    fsrv = (a4*np.log(-1/x)+b4*x+c4)
    rsrv = (a5*np.log(-1/x)+b5*x+c5)

    
    
    par_input = 10*np.stack((emitter_doping,back_doping,tau,fsrv,rsrv),axis=-1)
    if sim == 0 :
        unnorm_par = scaler.inverse_transform(par_input)
        return par_input,unnorm_par
        
    sim_curves= reg.predict(par_input)
           
  
    return sim_curves, par_input



In [ ]:
###############-Mean-of-most-probable-values-across-all-walkers-###############

In [ ]:
walker_maps = []

for w in range(samples.shape[1]):  # loop over walkers
    log_probs_walker = log_probs[:, w]  # shape: (n_steps,)
    samples_walker = samples[:, w, :]   # shape: (n_steps, ndim)

    # Find index of maximum log-probability for this walker
    idx_map = np.argmax(log_probs_walker)
    
    # Store corresponding parameter vector
    walker_maps.append(samples_walker[idx_map])

# Convert to NumPy array (shape: n_walkers × ndim)
walker_maps = np.array(walker_maps)

# Now take mean across walkers
mean_map_across_walkers = np.mean(walker_maps, axis=0)  # shape: (ndim,)

In [ ]:
print(mean_map_across_walkers )

In [ ]:
x_step = np.linspace(min(x), max(x), 50)
sim_JVs, _ = check_plot(mean_map_across_walkers,x,1)
fig, ax = plt.subplots(5, 1, figsize=(8, 10))
for i in range(5):
    ax[i].plot(sim_JVs[i, :], '--', label='Simulated')
    ax[i].plot(JV_exp[i, :], label='Experimental')
    ax[i].set_ylabel(f'Curve {i+1}')
    ax[i].legend()
ax[-1].set_xlabel("Voltage Index (or Time)")
plt.suptitle("Simulated vs Experimental JV Curves")
plt.tight_layout()
plt.show()


In [ ]:
_, par_input = check_plot(mean_map_across_walkers, x_step, 0)
par_in = np.exp(par_input)
def plot_uncertain_2(x,y):
    plt.plot(x,y,color='black')

plt.rcParams["figure.figsize"] = [8, 10]
plt.rcParams.update({'font.size': 16})
 
fig = plt.figure()
y_label = ['Conc.[cm-3]','Conc.[cm-3]', r'$\tau$ [s]', 'SRV [cm/S]','SRV [cm/S]']
x_labels = ['-1/530' ,'-1/580','-1/630','-1/680']
title = ['Zn emitter doping' , 'Si base doping' ,'bulk lifetime','Front SRV', 'Rear SRV']

for i in range(5):
    plt.subplot(5,1,i+1)
    
    l1=plot_uncertain_2(x_step,par_in[:,i]) 
    plt.yscale('log') 
    plt.ylabel(y_label[i])
    plt.xticks([-1000/530,-1000/580,-1000/630,-1000/680],[])
    plt.title(title[i],fontsize=15,fontweight='bold')
    plt.xlim(-1000/530,-1000/680)
    
  
plt.xticks([-1000/530,-1000/580,-1000/630,-1000/680], x_labels)

plt.xlabel(r'-1/T [1/C]') 

fig.align_labels()

In [ ]:
###############-Most-Probable-index-###############

In [ ]:
idx_map = np.argmax(flat_log_probs)
theta_last = flat_samples[idx_map, :]  
sim_JVs, _ = check_plot(theta_last, x, 1)  

fig, ax = plt.subplots(5, 1, figsize=(8, 10))
for i in range(5):
    ax[i].plot(sim_JVs[i, :], '--', label='Simulated')
    ax[i].plot(JV_exp[i, :], label='Experimental')
    ax[i].set_ylabel(f'Curve {i+1}')
    ax[i].legend()
ax[-1].set_xlabel("Voltage Index (or Time)")
plt.suptitle("Simulated vs Experimental JV Curves")
plt.tight_layout()
plt.show()

In [ ]:
x_step = np.linspace(min(x), max(x), 50)
_, par_input = check_plot(flat_samples[-1,:], x_step, 0)
par_in = np.exp(par_input)

def plot_uncertain_2(x,y):
    plt.plot(x,y,color='black')

plt.rcParams["figure.figsize"] = [8, 10]
plt.rcParams.update({'font.size': 16})
 
fig = plt.figure()
y_label = ['Conc.[cm-3]','Conc.[cm-3]', r'$\tau$ [s]', 'SRV [cm/S]','SRV [cm/S]']
x_labels = ['-1/530' ,'-1/580','-1/630','-1/680']
title = ['Zn emitter doping' , 'Si base doping' ,'bulk lifetime','Front SRV', 'Rear SRV']

for i in range(5):
    plt.subplot(5,1,i+1)
    
    l1=plot_uncertain_2(x_step,par_in[:,i]) 
    plt.yscale('log') 
    plt.ylabel(y_label[i])
    plt.xticks([-1000/530,-1000/580,-1000/630,-1000/680],[])
    plt.title(title[i],fontsize=15,fontweight='bold')
    plt.xlim(-1000/530,-1000/680)
    
  
plt.xticks([-1000/530,-1000/580,-1000/630,-1000/680], x_labels)

plt.xlabel(r'-1/T [1/C]') 

fig.align_labels()

In [ ]:
##############-Mean-of-all-values-###############

In [ ]:
idx_map = np.argmax(flat_log_probs)
theta_last = flat_samples_2[-1, :]  # Shape: (15,)
sim_JVs, _ = check_plot(theta_last, x, 1)  # x is your experimental temperature grid

# ---- Plot simulated vs experimental JV curves ----

fig, ax = plt.subplots(5, 1, figsize=(8, 10))
for i in range(5):
    ax[i].plot(sim_JVs[i, :], '--', label='Simulated')
    ax[i].plot(JV_exp[i, :], label='Experimental')
    ax[i].set_ylabel(f'Curve {i+1}')
    ax[i].legend()
ax[-1].set_xlabel("Voltage Index (or Time)")
plt.suptitle("Simulated vs Experimental JV Curves")
plt.tight_layout()
plt.show()

# ---- Extract material properties over a finer x grid ----

x_step = np.linspace(min(x), max(x), 50)

par_in = []
for i in range(flat_samples_2.shape[0]):
    try:
        _, par_input = check_plot(flat_samples_2[i, :], x_step, 0)
        if par_input.shape == (len(x_step), 5):
            par_in.append(par_input)
    except Exception as e:
        print(f"Skipping sample {i}: {e}")

par_in = np.array(par_in)  # Shape: (n_samples, 50, 5)
print(par_in.shape)
# ---- Discard burn-in (optional) ----
if par_in.shape[0] > 2000:
    par_in = par_in[2000:, :, :]

# ---- Convert from log-space to actual values ----
par_in = np.exp(par_in)  # Final shape: (n_good_samples, 50, 5)





In [ ]:
def plot_uncertain(x,y):
    
    mu = np.mean(y,axis = 0)
    std = np.std(y, axis = 0)
    plt.fill_between(x, mu+std,mu-std,alpha=0.1,color='grey')
    plt.plot(x,mu,color='black')

plt.rcParams["figure.figsize"] = [8, 10]
plt.rcParams.update({'font.size': 16})
 
fig = plt.figure()
y_label = ['Conc.[cm-3]','Conc.[cm-3]', r'$\tau$ [s]', 'SRV [cm/S]','SRV [cm/S]']
x_labels = ['-1/530' ,'-1/580','-1/630','-1/680']
title = ['Zn emitter doping' , 'Si base doping' ,'bulk lifetime','Front SRV', 'Rear SRV']


for i in range(5):
    plt.subplot(5,1,i+1)
    
    l1=plot_uncertain(x_step,par_in[:,:,i]) 
   
    plt.yscale('log') 
    plt.ylabel(y_label[i])
    plt.xticks([-1000/530,-1000/580,-1000/630,-1000/680],[])
    plt.title(title[i],fontsize=15,fontweight='bold')
    plt.xlim(-1000/530,-1000/680)
    
  
plt.xticks([-1000/530,-1000/580,-1000/630,-1000/680], x_labels)

plt.xlabel(r'-1/T [1/C]') 

fig.align_labels()

In [ ]:
for i in range(5):
    plt.subplot(5,1,i+1)
    plt.plot(samples[:, :, i])